In [1]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


# Selecting a Region 

## Cone Search

In [3]:
query = """
SELECT
TOP 10 source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
POINT(ra, dec),
CIRCLE(266.4, -29.0, 0.08))"""

POINT = ra and dec from individual rows in the table  
CIRCLE = circle center, radius  
CONTAINS: checks each point to see if its in the CIRCLE, returns 1 if it is  
    
In Where statement, set equal to 1 so returns all cases where CONTAINS returns 1


In [4]:
job = Gaia.launch_job(query)

result = job.get_results()
result

source_id
int64
4057468287575835392
4057482027171038976
4057470349160630656
4057469868125641984
4057468351995073024
4057470520960672640
4057470555320409600
4057482954883979392
4057470379220379776


Exercise 1: We've been using TOP to limit the size of queries returned - but this doesn't tell you how many rows your query actually returns. 
    The COUNT command returns the number of rows that would have been selected but does not return them.
In the previous query replace TOP with COUNT(source_id) and run the query again. How many stars has Gaia identified in the cone we searched?

In [6]:
query = """
SELECT
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
POINT(ra, dec),
CIRCLE(266.4, -29.0, 0.08))"""

In [9]:
job_ex1 = Gaia.launch_job(query)
result_ex1 = job_ex1.get_results()
result_ex1

count
int64
1658


# Working with Coordinates

Quantities object: number and unit (e.g. 30 deg is 30 + unit: deg)

In [10]:
import astropy.units as u

In [11]:
dir(u)

['A',
 'AA',
 'AB',
 'ABflux',
 'ABmag',
 'AU',
 'Angstrom',
 'B',
 'Ba',
 'Barye',
 'Bi',
 'Biot',
 'Bol',
 'Bq',
 'C',
 'Celsius',
 'Ci',
 'CompositeUnit',
 'D',
 'Da',
 'Dalton',
 'Debye',
 'Decibel',
 'DecibelUnit',
 'Dex',
 'DexUnit',
 'EA',
 'EAU',
 'EB',
 'EBa',
 'EC',
 'ED',
 'EF',
 'EG',
 'EGal',
 'EH',
 'EHz',
 'EJ',
 'EJy',
 'EK',
 'EL',
 'EN',
 'EOhm',
 'EP',
 'EPa',
 'ER',
 'ERy',
 'ES',
 'ESt',
 'ET',
 'EV',
 'EW',
 'EWb',
 'Ea',
 'Eadu',
 'Earcmin',
 'Earcsec',
 'Eau',
 'Eb',
 'Ebarn',
 'Ebeam',
 'Ebin',
 'Ebit',
 'Ebyte',
 'Ecd',
 'Echan',
 'Ecount',
 'Ect',
 'Ed',
 'Edeg',
 'Edyn',
 'EeV',
 'Eerg',
 'Eg',
 'Eh',
 'EiB',
 'Eib',
 'Eibit',
 'Eibyte',
 'Ek',
 'El',
 'Elm',
 'Elx',
 'Elyr',
 'Em',
 'Emag',
 'Emin',
 'Emol',
 'Eohm',
 'Epc',
 'Eph',
 'Ephoton',
 'Epix',
 'Epixel',
 'Erad',
 'Es',
 'Esr',
 'Eu',
 'Evox',
 'Evoxel',
 'Eyr',
 'F',
 'Farad',
 'Fr',
 'Franklin',
 'FunctionQuantity',
 'FunctionUnitBase',
 'G',
 'GA',
 'GAU',
 'GB',
 'GBa',
 'GC',
 'GD',
 'GF',
 '

### Creating a unit

In [14]:
ra = 30*u.deg
type(ra)

astropy.units.quantity.Quantity

In [15]:
ra

<Quantity 30. deg>

## Selecting the rectangle around GD-1

define x and y min and max values (subset to make dataset size more managable)

In [17]:
phi1_min = -55
phi1_max = -45
phi2_min = -8
phi2_max = 4

Build list of x and y coordinates of the vertices of a rectangle (CCW from lower left corner)

In [19]:
phi1_rect = [phi1_min, phi1_min, phi1_max, phi1_max]*u.deg
phi2_rect = [phi2_min, phi2_max, phi2_max, phi2_min]*u.deg

Figure in GD-1 frame (rotated to align with stream axes). Gaia catalog in ICRS coordinates (standard). To select these stars in the Gaia catalog we need to convert our vertices to from a rectangle in GD-1 space to a polygon in ICRS space

In [20]:
import gala.coordinates as gc

In [21]:
corners = gc.GD1Koposov10(phi1 = phi1_rect, phi2 = phi2_rect)
type(corners)

gala.coordinates.gd1.GD1Koposov10

In [22]:
corners

<GD1Koposov10 Coordinate: (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.)]>

In [23]:
import astropy.coordinates as coord

In [24]:
corners_icrs = corners.transform_to(coord.ICRS)

In [25]:
corners_icrs

<ICRS Coordinate: (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254)]>

# Selecting a polygon

#### Recall our example with CONTAINS

In [26]:
query = """
SELECT
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
POINT(ra, dec),
CIRCLE(266.4, -29.0, 0.08))"""

We're going to replace CIRLE with POLYGON in our contains example

POLYGON accepts a list of x,y values of the vertices

corners_icrs is a list of ra, dec tuples. We are first going to create a list of strings x, y, then join them together into a single string with the .join function

In [28]:
for point in corners_icrs:
    print(point.ra, point.dec)

146d16m31.1993s 19d15m42.8754s
135d25m17.902s 25d52m38.594s
141d36m09.5337s 34d18m17.3891s
152d49m00.1576s 27d08m10.0051s


In [31]:
for point in corners_icrs:
    print(point.ra.value, point.dec.value)

146.27533313607782 19.261909820533692
135.42163944306296 25.877387227672134
141.60264825107333 34.304830296257144
152.81671044675923 27.136112541397996


In [36]:
point_base = "{point.ra.value}, {point.dec.value}"
t = [point_base.format(point=point) 
     for point in corners_icrs]
t

['146.27533313607782, 19.261909820533692',
 '135.42163944306296, 25.877387227672134',
 '141.60264825107333, 34.304830296257144',
 '152.81671044675923, 27.136112541397996']

In [38]:
point_list = ', '.join(t)
point_list

'146.27533313607782, 19.261909820533692, 135.42163944306296, 25.877387227672134, 141.60264825107333, 34.304830296257144, 152.81671044675923, 27.136112541397996'

Notice the syntax for join is that it acts on the thing doing the joining and you pass it what needs to be joined

Now we're ready to build our query but we need to remake the columns we want to return

In [39]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity'

In [40]:
query_base = """SELECT {columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
AND bp_rp BETWEEN -0.75 AND 2
AND 1 = CONTAINS(POINT(ra, dec),
                 POLYGON({point_list}))"""

In [41]:
query = query_base.format(columns=columns, point_list=point_list)

In [42]:
print(query)

SELECT source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity
FROM gaiadr2.gaia_source
WHERE parallax < 1
AND bp_rp BETWEEN -0.75 AND 2
AND 1 = CONTAINS(POINT(ra, dec),
                 POLYGON(146.27533313607782, 19.261909820533692, 135.42163944306296, 25.877387227672134, 141.60264825107333, 34.304830296257144, 152.81671044675923, 27.136112541397996))


In [44]:
job = Gaia.launch_job_async(query)
print(job)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140340>
      name       dtype    unit                              description                             n_bad 
--------------- ------- -------- ------------------------------------------------------------------ ------
      source_id   int64          Unique source identifier (unique within a particular Data Release)      0
             ra float64      deg                                                    Right ascension      0
            dec float64      deg                                                        Declination      0
           pmra float64 mas / yr                         Proper motion in right ascension direction      0
          pmdec float64 mas / yr                             Proper motion in declination direction      0
       parallax float64      mas                                                           Parallax      0
 parallax_error float64      mas                                        

In [45]:
results = job.get_results()
len(results)

140340

This polygon selection has narrowed our results from 10 million rows (which Gaia won't even let us retrieve) to ~150k - 

# Saving Results

* substantial but managable dataset --> save
* can shutdown notebook and pick up where we left off
* use astropy Tables to write a fits file

In [46]:
filename = 'gd1_results.fits'
results.write(filename, overwrite=True)

.fits extension tells astropy to autoformat as fits file  
overwrite=True automatically overwrites the file

In [48]:
!ls -lh gd1_results.fits
#!dir gd1_results.fits

-rw-r--r--  1 bostroem  staff   8.6M Nov 16 19:16 gd1_results.fits
